In [27]:
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from Mesh.Charges_utils import get_charges_list
from Model.Solutions_utils import Solution_utils

from Mesh.Mesh import Domain_Mesh
from Post.Postcode import Postprocessing as Post
from Simulation import get_simulation_name

simulation_name,results_path,main_path,logger = get_simulation_name(os.path.join(os.path.dirname(os.getcwd()), 'dataset.py'))

file_path = os.getcwd()

In [30]:

molecule = 'methanol_E'


Rs = {'born_ion': 1,
      'methanol': 2.54233,
      'methanol_E': 2.54233,
      'arg': 5.9695745,
      'sphere': 1.2,
      'sphere_+1-1': 1.2,
      'sphere_+1+1': 1.2,
      'sphere_+2-1': 1.2,
      'sphere_+1_D': 1.2,
      'sphere_0.2': 1.2}

Model = Solution_utils()

Model.pqr_path = os.path.join(os.getcwd(),molecule,f'{molecule}.pqr')
q_list = get_charges_list(Model.pqr_path)

Model.domain_properties = {
        'molecule': molecule,
        'epsilon_1':  1,
        'epsilon_2': 80,
        'kappa': 0.125,
        'T' : 300 
        }

Model.epsilon_1 = Model.domain_properties['epsilon_1']
Model.epsilon_2 = Model.domain_properties['epsilon_2']
Model.kappa = Model.domain_properties['kappa']
Model.pi = np.pi
Model.q_list = q_list

n = len(Model.q_list)
Model.qs = np.zeros(n)
Model.x_qs = np.zeros((n,3))
for i,q in enumerate(Model.q_list):
    Model.qs[i] = q.q
    Model.x_qs[i,:] = q.x_q
Model.total_charge = np.sum(Model.qs)

dR = 0.1
R_mol = Rs[molecule]
Rmin = dR + R_mol
Rmax = Rmin + 3

Model.mesh_density = 3
Model.mesh_generator = 'msms'
Model.DTYPE = 'float64'

In [31]:
q_list

[Charges(q=1.0, x_q=array([0.019 , 0.    , 0.6556]), r_q=1.8875, atom_name='CH3', res_name='MEOH', res_num='1', ion_r_explode=3.5),
 Charges(q=-2.59, x_q=array([-0.1197,  0.    , -0.7372]), r_q=1.535, atom_name='O', res_name='MEOH', res_num='1', ion_r_explode=3.5),
 Charges(q=1.5926, x_q=array([ 0.737,  0.   , -1.139]), r_q=0.2, atom_name='H', res_name='MEOH', res_num='1', ion_r_explode=3.5)]

### Calculate Solvation energy

In [32]:
phi_q = Model.Harmonic_spheres(Model.x_qs,'molecule',R_mol)
print(Model.solvation_energy_phi_qs(phi_q))
print(phi_q)
Model.solvation_energy_phi_qs(phi_q)

-46.480951306374806
[-0.00274579  0.00353542 -0.00651473]


-46.480951306374806

### Create data_known file

In [33]:

def calculate_print_phi(domain,method, num_points, file):

    R = R_mol if domain == 'molecule' else Rmax if domain == 'solvent' else None
    x = np.random.uniform(-R, R, num_points)
    y = np.random.uniform(-R, R, num_points)
    z = np.random.uniform(-R, R, num_points)
    r = np.sqrt(x**2+y**2+z**2)
    X = np.stack([x,y,z], axis=1)

    phi_values = Model.phi_known(method,'phi',X,domain,R_mol)

    if domain == 'molecule':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]<R_mol:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

    elif domain == 'solvent':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]>Rmin:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

In [35]:
with open(os.path.join(file_path,molecule,'data_known.dat'), 'w') as file:
    calculate_print_phi('solvent','pbj',300, file)
    # calculate_print_phi('molecule','Harmonic_spheres',300, file)

print('File created')

File created
